# 5-7-自己回帰モデルとその周辺

In [ ]:
# -*- coding: utf-8 -*-
# 5-7 自己回帰モデル（R+Stan → Python+NumPyro 版）

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive
from numpyro.contrib.render import render_model

In [ ]:
# ----------------------------------------------------------------------
# 0) Settings (seed, print precision)
# ----------------------------------------------------------------------
rng_key = jax.random.PRNGKey(1)  # Rの seed=1 に対応
np.set_printoptions(suppress=True, linewidth=120)

In [ ]:
# ----------------------------------------------------------------------
# 1) Load CSV with pandas (no .loc is used)
# ----------------------------------------------------------------------
# ファイル名は元コードに合わせています
df = pd.read_csv("5-7-1-sales-ts-5.csv", parse_dates=["date"])
print("Head (first 3 rows):")
print(df.head(3))  # 計算結果の表示は print() を使用

# 観測系列
y_np = df["sales"].to_numpy(dtype=np.float32)
T = int(len(df))  # numpyro.plate の N は Python int にする

In [ ]:
# ----------------------------------------------------------------------
# 2) Quick visualization (matplotlib)
#    ※ ラベルは英語
# ----------------------------------------------------------------------
plt.figure(figsize=(8, 3.5))
plt.plot(df["date"].to_numpy(), df["sales"].to_numpy())
plt.title("Sales Time Series")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------------------------------------------------
# 3) AR(1) model in NumPyro  (Stanコードの写経)
#    y[i] ~ Normal(Intercept + b_ar * y[i-1], s_w)  (i=2..T)
#    ※ Stanでは暗黙事前でしたがNumPyroでは明示的に弱情報事前を置きます
# ----------------------------------------------------------------------
def ar1_model(y=None, T=None):
    s_w = numpyro.sample("s_w", dist.HalfNormal(10.0))     # process noise sd
    b_ar = numpyro.sample("b_ar", dist.Normal(0.0, 1.0))   # AR(1) coeff
    Intercept = numpyro.sample("Intercept", dist.Normal(0.0, 10.0))  # intercept

    # 観測は y[1:], 平均は Intercept + b_ar * y[:-1]
    # ループ i=2..T に対応して長さ (T-1)
    N = int(T) - 1
    with numpyro.plate("time", N):
        mu = Intercept + b_ar * y[:-1]
        numpyro.sample("y", dist.Normal(mu, s_w), obs=y[1:] if y is not None else None)

In [ ]:
# ----------------------------------------------------------------------
# 4) Visualize the Bayesian model structure with NumPyro's built-in function
#    （ベイズ統計モデルの可視化はNumPyro組み込みを使用）
# ----------------------------------------------------------------------
try:
    g = render_model(ar1_model, model_kwargs={"y": y_np, "T": T}, render_distributions=True)
    # Graphviz が入っていればファイル出力も可能（任意）
    g.render(filename="autoregressive_model", format="png", cleanup=True)
    print("Rendered model graph to 'autoregressive_model.png'")
except Exception as e:
    print(f"Model rendering skipped (Graphviz not available?): {e}")

In [ ]:
# ----------------------------------------------------------------------
# 5) MCMC (NUTS) with NumPyro
#    Stanの control(list(max_treedepth=15)) に対応
# ----------------------------------------------------------------------
nuts = NUTS(ar1_model, max_tree_depth=15)
mcmc = MCMC(nuts, num_warmup=1000, num_samples=2000, num_chains=4, progress_bar=True)
mcmc.run(rng_key, y=y_np, T=T)

# mcmc.postprocess_samples()  # ← 削除

# チェーン別サンプルを NumPy に転送して取得
posterior_chain = jax.device_get(mcmc.get_samples(group_by_chain=True))

In [ ]:
# ----------------------------------------------------------------------
# 6) Convert to ArviZ InferenceData (禁止事項に合わせ az.from_dict を使用)
#    - az.from_numpyro の mcmc / sample_stats / observed_data 引数は使用しない
#    - coords/dims を与えて後でPPCを重ねやすくする
# ----------------------------------------------------------------------
coords = {"time": np.arange(1, T, dtype=int)}   # y のインデックス(2..T)に対応
dims = {"y": ["time"]}

idata = az.from_dict(
    posterior=posterior_chain,   # posterior samples（チェーン結合せず）
    coords=coords,
    dims=dims,
)

# 概要統計 (print() で表示)
print("\nMCMC summary (posterior):")
print(az.summary(idata, var_names=["s_w", "b_ar", "Intercept"]))

# Gelman-Rubin などは summary に含まれる r_hat を参照
# 追加でダイバージェンス数も確認（print）
try:
    diverging = mcmc.get_extra_fields()["diverging"]
    n_div = int(np.asarray(diverging).sum())
    print(f"\nNumber of divergent transitions: {n_div}")
except Exception as e:
    print(f"\nDivergence check skipped: {e}")

In [ ]:
# ----------------------------------------------------------------------
# 7) Posterior plots with ArviZ
#    - 事後分布の可視化に ArviZ を使用
#    - az.plot_posterior(..., hdi_prob=...) を使用（credible_interval は不使用）
#    - トレースプロットも参考として作成
# ----------------------------------------------------------------------
az.plot_trace(idata, var_names=["s_w", "b_ar", "Intercept"])
plt.tight_layout()
plt.show()

az.plot_posterior(idata, var_names=["s_w", "b_ar", "Intercept"], hdi_prob=0.95)
plt.tight_layout()
plt.show()

# HDI（戻り値 Dataset を用い、変数名で取り出す）
hdi_ds = az.hdi(idata, hdi_prob=0.95)
print("\n95% HDI for b_ar:")
print(hdi_ds["b_ar"])

# 森林図（group 引数は使用禁止のため未指定）
az.plot_forest(idata, var_names=["s_w", "b_ar", "Intercept"], combined=True)
plt.title("Forest Plot (Combined Chains)")
plt.show()

In [ ]:
# ----------------------------------------------------------------------
# 8) Posterior Predictive Check (PPC)
#    - Predictive の batch_ndims=1（チェイン結合を示す 1）を必ず指定
#    - 観測 y を条件に y[1:] の事後予測を生成（mu は y[:-1] に依存）
#    - az.plot_ppc(..., group="posterior")
# ----------------------------------------------------------------------
rng_key, rng_pp = jax.random.split(rng_key)
# 1) 事後サンプル（チェイン別）
posterior_chain = jax.device_get(mcmc.get_samples(group_by_chain=True))  # (chains, draws, ...)

# 2) Posterior predictive
predictive = Predictive(
    ar1_model,
    posterior_samples=posterior_chain,
    return_sites=["y"],
    batch_ndims=2,  # ← (chain, draw) の2次元をバッチ化
)
pp = predictive(rng_pp, y=y_np, T=T)       # pp["y"]: (chains, draws, time)

# 3) ArviZ へ
idata_ppc = az.from_dict(
    posterior=posterior_chain,
    posterior_predictive={"y": np.asarray(pp["y"])},  # (chains, draws, time)
    observed_data={"y": y_np[1:]},
    coords=coords,
    dims=dims,  # dims={"y": ["time"]}
)


# PPC プロット（group="posterior" を必ず渡す）
az.plot_ppc(idata_ppc, group="posterior", num_pp_samples=100)
plt.title("Posterior Predictive Check (AR(1))")
plt.xlabel("y index (t = 2..T)")
plt.ylabel("Sales")
plt.show()